# 문제 1. 
- 다음은 E-Commerce Shipping 데이터 세트이다. 주어진 훈련 데이터 세트를 활용하여 고객이 주문한 물품의 정시 도착 여부를 예측하고 해당 예측 결과를 csv 파일로 제출하시오.
- 결과 예측 양식 : 제출한 예측값의 ROC_AUC 점수 결과를 통해 영역별 배점에 따라 최종 점수가 반영될 예정

In [1]:
import pandas as pd 
pd.DataFrame({
    "ID"  : [0, 1, 2], 
    "y_pred" : [0.2543, 0.1324, 0.5892]
})

,ID,y_pred
0,0,0.2543
1,1,0.1324
2,2,0.5892


In [2]:
import pandas as pd 
X_test = pd.read_csv("data/2회/204_x_test.csv")
X_train = pd.read_csv("data/2회/204_x_train.csv")
y_train = pd.read_csv("data/2회/204_y_train.csv")

## 데이터 정보

In [3]:
import pandas as pd 
pd.DataFrame({
    "변수" : list(X_train.columns) + ["Reached.on.Time_Y.N"], 
    "설명" : [
        "고객 ID 번호", 
        "창고의 블록 단위 구역(A, B, C, D, F)", 
        "제품 배송 방법", 
        "문의 전화 수", 
        "고객의 등급(1: 가장 낮음, 5: 가장 높음)", 
        "제품의 비용(달러 기준)",
        "사전 구매 수량", 
        "제품의 중요도", 
        "성별(F: 여성, M: 남성)", 
        "할인혜택", 
        "그램 단위 무게", 
        "정시 도착 여부(1: 도착하지 않음, 0: 정시 도착)"
    ]
})

,변수,설명
0,ID,고객 ID 번호
1,Warehouse_block,"창고의 블록 단위 구역(A, B, C, D, F)"
2,Mode_of_Shipment,제품 배송 방법
3,Customer_care_calls,문의 전화 수
4,Customer_rating,"고객의 등급(1: 가장 낮음, 5: 가장 높음)"
5,Cost_of_the_Product,제품의 비용(달러 기준)
6,Prior_purchases,사전 구매 수량
7,Product_importance,제품의 중요도
8,Gender,"성별(F: 여성, M: 남성)"
9,Discount_offered,할인혜택


## 데이터확인

In [4]:
X_train.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,9902,F,Ship,3,5,214,2,medium,F,6,4578
1,9501,A,Ship,4,2,201,3,medium,F,4,4613
2,6111,A,Ship,4,2,264,3,low,M,10,5137
3,588,F,Ship,3,5,194,2,medium,M,4,3496
4,10806,F,Road,5,5,269,5,medium,F,3,1120


In [5]:
y_train.head()

,ID,Reached.on.Time_Y.N
0,9902,0
1,9501,0
2,6111,1
3,588,1
4,10806,0


In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6599 entries, 0 to 6598
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   6599 non-null   int64 
 1   Warehouse_block      6599 non-null   object
 2   Mode_of_Shipment     6599 non-null   object
 3   Customer_care_calls  6599 non-null   int64 
 4   Customer_rating      6599 non-null   int64 
 5   Cost_of_the_Product  6599 non-null   int64 
 6   Prior_purchases      6599 non-null   int64 
 7   Product_importance   6599 non-null   object
 8   Gender               6599 non-null   object
 9   Discount_offered     6599 non-null   int64 
 10  Weight_in_gms        6599 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 567.2+ KB


In [7]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6599 entries, 0 to 6598
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   ID                   6599 non-null   int64
 1   Reached.on.Time_Y.N  6599 non-null   int64
dtypes: int64(2)
memory usage: 103.2 KB


In [8]:
y_train['Reached.on.Time_Y.N'].value_counts()

1    3938
0    2661
Name: Reached.on.Time_Y.N, dtype: int64

In [9]:
X_train.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [10]:
X_test.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [11]:
X_train.describe()

,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
count,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000,6599.000000
mean,5471.118503,4.058494,2.993181,209.587968,3.565086,13.236703,3638.925746
std,3160.586530,1.134313,1.415857,48.423993,1.533984,16.066606,1632.253471
min,3.000000,2.000000,1.000000,96.000000,2.000000,1.000000,1001.000000
25%,2755.500000,3.000000,2.000000,169.000000,3.000000,4.000000,1844.000000
50%,5424.000000,4.000000,3.000000,213.000000,3.000000,7.000000,4152.000000
75%,8201.000000,5.000000,4.000000,251.000000,4.000000,10.000000,5049.500000
max,10999.000000,7.000000,5.000000,310.000000,10.000000,65.000000,7401.000000


## 데이터셋 분리

In [12]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X_train, y_train, test_size = 0.2, random_state=42)
X_tr.shape, X_te.shape, y_tr.shape, y_te.shape

((5279, 11), (1320, 11), (5279, 2), (1320, 2))

## 컬럼 분리

In [13]:
X_tr_id = X_tr.pop('ID')
X_te_id = X_te.pop('ID')
y_tr.pop('ID')
y_te.pop('ID')
X_tr.shape, X_te.shape, y_tr.shape, y_te.shape

((5279, 10), (1320, 10), (5279, 1), (1320, 1))

In [14]:
X_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5279 entries, 589 to 860
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Warehouse_block      5279 non-null   object
 1   Mode_of_Shipment     5279 non-null   object
 2   Customer_care_calls  5279 non-null   int64 
 3   Customer_rating      5279 non-null   int64 
 4   Cost_of_the_Product  5279 non-null   int64 
 5   Prior_purchases      5279 non-null   int64 
 6   Product_importance   5279 non-null   object
 7   Gender               5279 non-null   object
 8   Discount_offered     5279 non-null   int64 
 9   Weight_in_gms        5279 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 453.7+ KB


## 데이터 타입별로 분리

In [15]:
import numpy as np
object_df = X_tr.select_dtypes(include=object)
object_df.head()

,Warehouse_block,Mode_of_Shipment,Product_importance,Gender
589,F,Ship,medium,F
2531,D,Ship,medium,F
5462,B,Road,low,M
3909,F,Ship,medium,F
3130,D,Road,medium,F


In [16]:
for column in object_df.columns:
    print(object_df[column].value_counts())

F    1783
A     904
B     875
D     859
C     858
Name: Warehouse_block, dtype: int64
Ship      3592
Flight     865
Road       822
Name: Mode_of_Shipment, dtype: int64
low       2514
medium    2310
high       455
Name: Product_importance, dtype: int64
F    2640
M    2639
Name: Gender, dtype: int64


In [17]:
numbers_df = X_tr.select_dtypes(include=np.number)
numbers_df.head()

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
589,4,3,274,3,8,4276
2531,6,1,286,7,7,2338
5462,4,2,227,5,7,1354
3909,3,1,232,2,2,5104
3130,2,3,215,2,3,5052


## 모형 학습 및 평가

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, OneHotEncoder, OrdinalEncoder
from lightgbm import LGBMClassifier
from sklearn.compose import ColumnTransformer

# 컬럼 변환기 설정
column_transformer = ColumnTransformer([
    ('scaler', MinMaxScaler(), numbers_df.columns),
    ('ohc_encoder', OneHotEncoder(), ['Warehouse_block', 'Mode_of_Shipment', 'Gender']),
    ('ordinal_encoder', OrdinalEncoder(categories=[['low', 'medium', 'high']]), ['Product_importance']) # object값의 순서에 의미가 있는 경우 사용
], remainder='passthrough')

# 파이프라인 설정하여 전처리 및 모델 설정
pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('classifier', LGBMClassifier(random_state=42, max_depth=5, learning_rate=0.001))
])

# 모형학습
pipeline.fit(X_tr, y_tr)

D:\Anaconda\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', MinMaxScaler(),
                                                  Index(['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product',
       'Prior_purchases', 'Discount_offered', 'Weight_in_gms'],
      dtype='object')),
                                                 ('ohc_encoder',
                                                  OneHotEncoder(),
                                                  ['Warehouse_block',
                                                   'Mode_of_Shipment',
                                                   'Gender']),
                                                 ('ordinal_encoder',
                                                  OrdinalEncoder(categories=[['low',
                                                                              'medium',
                                                                              'high']]),
                                                  ['Product_importance'])])),
                ('classifier',
                 LGBMClassifier(learning_rate=0.001, max_depth=5,
                                random_state=42))])

## 모형 평가

In [23]:
from sklearn.metrics import roc_auc_score

def get_scores(model, X_tr, X_te, y_tr, y_te):
    train_predict = model.predict_proba(X_tr)
    test_predict = model.predict_proba(X_te)
    train_score = roc_auc_score(y_tr, train_predict[:, 1])
    test_score = roc_auc_score(y_te, test_predict[:, 1])
    return f'train : {train_score}, test : {test_score}'

get_scores(pipeline, X_tr, X_te, y_tr, y_te)

'train : 0.7668840188239993, test : 0.7522266555727845'

## 결과 제출

In [24]:
predictions = pipeline.predict_proba(X_te)
result = pd.DataFrame({'ID' : X_te_id, "pred": predictions[:, 1]})
result

,ID,pred
2098,1901,0.637836
5338,9142,0.586370
4407,3033,0.621973
6108,2737,0.637836
4025,10848,0.579040
...,...,...
3268,1099,0.637836
3949,1987,0.637836
3077,2814,0.637836
3767,4278,0.616343


In [25]:
result.to_csv('1.csv', index=False)